In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress


In [2]:
movies = Path('resources/movies_metadata.csv')


movie_df = pd.read_csv(movies, low_memory=False)


movie_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
clean_movie = movie_df[['title','budget','revenue','release_date','vote_average','genres']]
clean_movie

,title,budget,revenue,release_date,vote_average,genres
0,Toy Story,30000000,373554033.0,1995-10-30,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,65000000,262797249.0,1995-12-15,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,0,0.0,1995-12-22,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,16000000,81452156.0,1995-12-22,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,0,76578911.0,1995-02-10,5.7,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...,...,...
45461,Subdue,0,0.0,NaN,4.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
45462,Century of Birthing,0,0.0,2011-11-17,9.0,"[{'id': 18, 'name': 'Drama'}]"
45463,Betrayal,0,0.0,2003-08-01,3.8,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,Satan Triumphant,0,0.0,1917-10-21,0.0,[]


In [4]:
cleaned = clean_movie.rename(columns={
    'release_date':'date',
    'vote_average':'rating'
})
cleaned.count()

title      45460
budget     45466
revenue    45460
date       45379
rating     45460
genres     45466
dtype: int64

In [5]:
#dropping any null values
cleaner = cleaned.dropna(how='any')
cleaner = cleaner[~cleaner[list(('budget','revenue', 'rating'))].eq(0).all(axis = 1)]
cleaner

,title,budget,revenue,date,rating,genres
0,Toy Story,30000000,373554033.0,1995-10-30,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,65000000,262797249.0,1995-12-15,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,0,0.0,1995-12-22,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,16000000,81452156.0,1995-12-22,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,0,76578911.0,1995-02-10,5.7,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...,...,...
45460,Robin Hood,0,0.0,1991-05-13,5.7,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name..."
45462,Century of Birthing,0,0.0,2011-11-17,9.0,"[{'id': 18, 'name': 'Drama'}]"
45463,Betrayal,0,0.0,2003-08-01,3.8,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,Satan Triumphant,0,0.0,1917-10-21,0.0,[]


In [7]:
#wanted to pull all the info from the 'genres' column to see what was going on.
max_genres = cleaner['genres'].str.count(',').max()+1
new_columns = [f'genre{i}' for i in range(1, max_genres +1)]
cleaner[new_columns] = cleaner['genres'].str.split(',', expand=True)
print(cleaner)

                             title    budget      revenue        date  rating  \
0                        Toy Story  30000000  373554033.0  1995-10-30     7.7   
1                          Jumanji  65000000  262797249.0  1995-12-15     6.9   
2                 Grumpier Old Men         0          0.0  1995-12-22     6.5   
3                Waiting to Exhale  16000000   81452156.0  1995-12-22     6.1   
4      Father of the Bride Part II         0   76578911.0  1995-02-10     5.7   
...                            ...       ...          ...         ...     ...   
45460                   Robin Hood         0          0.0  1991-05-13     5.7   
45462          Century of Birthing         0          0.0  2011-11-17     9.0   
45463                     Betrayal         0          0.0  2003-08-01     3.8   
45464             Satan Triumphant         0          0.0  1917-10-21     0.0   
45465                     Queerama         0          0.0  2017-06-09     0.0   

                           

In [8]:
#wanted to remove all the extra columns that held no info 
cleaner.drop(cleaner.columns[[5,12,13,14,15,16,17,18,19,20,21]], axis=1, inplace=True)
cleaner

,title,budget,revenue,date,rating,genre1,genre2,genre3,genre4,genre5,genre6
0,Toy Story,30000000,373554033.0,1995-10-30,7.7,[{'id': 16,'name': 'Animation'},{'id': 35,'name': 'Comedy'},{'id': 10751,'name': 'Family'}]
1,Jumanji,65000000,262797249.0,1995-12-15,6.9,[{'id': 12,'name': 'Adventure'},{'id': 14,'name': 'Fantasy'},{'id': 10751,'name': 'Family'}]
2,Grumpier Old Men,0,0.0,1995-12-22,6.5,[{'id': 10749,'name': 'Romance'},{'id': 35,'name': 'Comedy'}],None,None
3,Waiting to Exhale,16000000,81452156.0,1995-12-22,6.1,[{'id': 35,'name': 'Comedy'},{'id': 18,'name': 'Drama'},{'id': 10749,'name': 'Romance'}]
4,Father of the Bride Part II,0,76578911.0,1995-02-10,5.7,[{'id': 35,'name': 'Comedy'}],None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
45460,Robin Hood,0,0.0,1991-05-13,5.7,[{'id': 18,'name': 'Drama'},{'id': 28,'name': 'Action'},{'id': 10749,'name': 'Romance'}]
45462,Century of Birthing,0,0.0,2011-11-17,9.0,[{'id': 18,'name': 'Drama'}],None,None,None,None
45463,Betrayal,0,0.0,2003-08-01,3.8,[{'id': 28,'name': 'Action'},{'id': 18,'name': 'Drama'},{'id': 53,'name': 'Thriller'}]
45464,Satan Triumphant,0,0.0,1917-10-21,0.0,[],None,None,None,None,None


In [10]:
#did the same as above but for 'date' column
max_dates = cleaner['date'].str.count('-').max()+1
max_dates
new_dates = [f'date{i}' for i in range(1, max_dates +1)]
cleaner[new_dates] = cleaner['date'].str.split('-', expand=True)
print(cleaner)

                             title    budget      revenue        date  rating  \
0                        Toy Story  30000000  373554033.0  1995-10-30     7.7   
1                          Jumanji  65000000  262797249.0  1995-12-15     6.9   
2                 Grumpier Old Men         0          0.0  1995-12-22     6.5   
3                Waiting to Exhale  16000000   81452156.0  1995-12-22     6.1   
4      Father of the Bride Part II         0   76578911.0  1995-02-10     5.7   
...                            ...       ...          ...         ...     ...   
45460                   Robin Hood         0          0.0  1991-05-13     5.7   
45462          Century of Birthing         0          0.0  2011-11-17     9.0   
45463                     Betrayal         0          0.0  2003-08-01     3.8   
45464             Satan Triumphant         0          0.0  1917-10-21     0.0   
45465                     Queerama         0          0.0  2017-06-09     0.0   

              genre1       

In [11]:
#it felt easier to just copy what I did above
cleaner.drop(cleaner.columns[[5,7,9,12,13]], axis=1, inplace=True)
cleaner

,title,budget,revenue,date,rating,genre2,genre4,genre6,date1
0,Toy Story,30000000,373554033.0,1995-10-30,7.7,'name': 'Animation'},'name': 'Comedy'},'name': 'Family'}],1995
1,Jumanji,65000000,262797249.0,1995-12-15,6.9,'name': 'Adventure'},'name': 'Fantasy'},'name': 'Family'}],1995
2,Grumpier Old Men,0,0.0,1995-12-22,6.5,'name': 'Romance'},'name': 'Comedy'}],None,1995
3,Waiting to Exhale,16000000,81452156.0,1995-12-22,6.1,'name': 'Comedy'},'name': 'Drama'},'name': 'Romance'}],1995
4,Father of the Bride Part II,0,76578911.0,1995-02-10,5.7,'name': 'Comedy'}],None,None,1995
...,...,...,...,...,...,...,...,...,...
45460,Robin Hood,0,0.0,1991-05-13,5.7,'name': 'Drama'},'name': 'Action'},'name': 'Romance'}],1991
45462,Century of Birthing,0,0.0,2011-11-17,9.0,'name': 'Drama'}],None,None,2011
45463,Betrayal,0,0.0,2003-08-01,3.8,'name': 'Action'},'name': 'Drama'},'name': 'Thriller'}],2003
45464,Satan Triumphant,0,0.0,1917-10-21,0.0,None,None,None,1917


In [24]:
#had to have some help with chatgpt to get rid of " 'name': " from the names in genre2 
#so it'll just read as the genre it is

cleaner['genre2'] = cleaner['genre2'].str.extract("'name': '([^']*)'")

cleaner.head()

,title,budget,revenue,date,rating,genre2,genre4,genre6,date1
0,Toy Story,30000000,373554033.0,1995-10-30,7.7,Animation,'name': 'Comedy'},'name': 'Family'}],1995
1,Jumanji,65000000,262797249.0,1995-12-15,6.9,Adventure,'name': 'Fantasy'},'name': 'Family'}],1995
2,Grumpier Old Men,0,0.0,1995-12-22,6.5,Romance,'name': 'Comedy'}],None,1995
3,Waiting to Exhale,16000000,81452156.0,1995-12-22,6.1,Comedy,'name': 'Drama'},'name': 'Romance'}],1995
4,Father of the Bride Part II,0,76578911.0,1995-02-10,5.7,Comedy,None,None,1995


In [42]:
#Wanted to drop genre4 and genre6 '(but not get rid of it from the data frame)' 
#and move date1 to the front as it should help with making the graphs and such

almost = cleaner[['date1','title','budget','revenue','rating','genre2']]
#wanted to change the name of the columns to make it a little bit easier to work with
close = almost.rename(columns={
    'date1':'year',
    'genre2':'genre'
})
#made sure to clear all the dates we dont want
scrubbed = close[close['year'].astype(int) > 1999]
juggle = scrubbed[scrubbed['year'].astype(int) < 2021]
juggle = juggle.sort_values('year', ascending=True)
juggle

,year,title,budget,revenue,rating,genre
4944,2000,Waydowntown,555000,0.0,6.0,Comedy
17462,2000,Hurmaava joukkoitsemurha,0,0.0,5.0,NaN
24230,2000,Hur som helst är han jävligt död,0,0.0,0.0,Comedy
33167,2000,Alaipayuthey,2000000,6000000.0,6.7,Romance
5289,2000,Gangster No. 1,0,0.0,6.5,Action
...,...,...,...,...,...,...
44535,2018,Mobile Homes,0,0.0,0.0,Drama
38130,2018,Mary Shelley,0,0.0,0.0,Drama
30402,2018,Bad Boys for Life,0,0.0,0.0,Thriller
33359,2018,Iron Sky: The Coming Race,18000000,0.0,0.0,Action


In [43]:
#not sure if we wanted to keep any of the columns as with values that are either zero or 0.0 or NaN
#so with this box of coding and lower are me removing all zero,0.0, or NaN values from the dataframe
columns_to_check = ['budget', 'revenue','genre'] 
scrubbed = scrubbed[~scrubbed[columns_to_check].astype(str).isin(['0', '0.0', np.nan]).any(axis=1)]
scrubbed

,year,title,budget,revenue,rating,genre
3060,2000,Next Friday,11000000,59827328.0,6.5,Comedy
3073,2000,Supernova,90000000,14828081.0,4.9,Horror
3121,2000,Isn't She Great,36000000,3003296.0,4.6,Drama
3155,2000,Scream 3,40000000,161834276.0,5.7,Horror
3158,2000,Gun Shy,10000000,1631839.0,5.4,Action
...,...,...,...,...,...,...
45250,2007,Sivaji: The Boss,12000000,19000000.0,6.9,Action
45399,2014,All at Once,750000,3.0,6.0,Crime
45409,2006,Savages,800000,1328612.0,5.8,Comedy
45412,2010,Pro Lyuboff,2000000,1268793.0,4.0,Romance
